In [89]:
# Imports
%load_ext autoreload
%autoreload 2

import sys
# sys.path.append('../')

import numpy as np

import matplotlib.pyplot as plt
import scipy.stats as stats

import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error

from dml4fluxes.experiments import experiment, experiment_utils
from dml4fluxes.analysis.visualization import *
from dml4fluxes.datasets.preprocessing import *
from dml4fluxes.experiments.utility import get_available_sites, get_igbp_of_site, transform_t, JSONEncoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
pre_computed=False
syn=True
relnoise=0
site = 'all'
dataset_config = dict(
            site_name = site,
            syn = syn,
            version = 'simple',
            Q10=1.5,
            relnoise=relnoise,
            pre_computed=pre_computed,
            transform_T = True,
            month_wise= False,
            moving_window=[3, 3],
            delta = 'heuristic3', #fixed value, max_heuristic, slope_heuristic 
            X = ['TA', 'SWC', 'RH', 'WS', 'WD'],
            #X = ['VPD', 'TA','wdefcum', 'SW_POT_sm', 'SW_POT_sm_diff', 'PA', 'WS', 'GPP_prox', 'P', 'WD_sin', 'WD_cos', 'NEE_nt_avg', 'TS_1', 'TS_2', 'SWC_1', 'SWC_2', 'SW_DIF'], #['VPD', 'TA'],#
            #X = ['VPD', 'TA', 'TS_1', 'TS_2', 'SWC_1', 'SWC_2'],
            W = [], #['SW_POT_sm', 'SW_POT_sm_diff'], #['wdefcum'],
            Q10_drivers = ['SW_IN'],
            years = 'all', # 'all'
            norm_type = 'standardize',
            Q10_fitting=False,
            Q10_include_SW=False,
            test_scenario=True,
            RMSE=True,
            alternative_fluxes=None,
            alternative_treatment=None, #"SW_IN_POT",
            )
    
#model_y_config = dict(model = 'GradientBoostingRegressor',
#                    min_samples_split = 5,
#                    min_samples_leaf = 40,
#                    max_depth=1,
#                    n_estimators=300,
#                    #learning_rate=0.1,
#                    #max_depth=3
#                    )

model_y_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 5,
                    min_samples_leaf = 40,
                    max_depth=1,
                    n_estimators=300,
                    #learning_rate=0.1,
                    #max_depth=3
                    )

model_t_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 5,
                    min_samples_leaf = 40,
                    max_depth=1,
                    n_estimators=300,
                    # max_depth=3
                    )
model_final_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 5,
                    min_samples_leaf = 40,
                    max_depth=2,
                    n_estimators=300)
                    #learning_rate=0.1,
                    #max_depth=3
dml_config = dict(cv=10)

model_configs = dict(y = model_y_config,
                    t = model_t_config,
                    final= model_final_config,
                    dml =dml_config)

experiment_config = dict(experiment_type = 'flux_partitioning',
                        comment='Flux partitioning',
                        extrapolate=False,
                        seed=1000)



In [143]:
dataset_1 = pd.read_csv('/home/kaicohrs/Repositories/dml-4-fluxes/data/NNinput_SCOPE_US_Ha1_1314.csv')

In [144]:
dataset_2 = pd.read_csv('/home/kaicohrs/Repositories/dml-4-fluxes/data/test_data_SCOPE.csv')

In [145]:
dataset_3 = pd.read_csv('/home/kaicohrs/Repositories/dml-4-fluxes/data/hf004-02-filled.csv')

In [146]:
dataset_4 = pd.read_csv('/home/kaicohrs/Repositories/dml-4-fluxes/data/hf004-01-final.csv')

In [148]:
for dataset in [dataset_2]:
    dataset['hour'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").hour)
    dataset['minute'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").minute)
    dataset['second'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").second)
    dataset['day'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%x").day)
    dataset['month'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%x").month)
    dataset['year'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%x").year)
    
    dataset['datetime'] = dataset.apply(lambda x: datetime.datetime(x['year'],
                                                                    x['month'],
                                                                    x['day'],
                                                                    x['hour'],
                                                                    x['minute'],
                                                                    x['second']),
                                                                    axis=1
                                                                    )
    

for dataset in [dataset_1]:
    dataset['hour'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").hour)
    dataset['minute'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").minute)
    dataset['second'] = dataset['Time'].apply(lambda x:datetime.datetime.strptime(x,"%H:%M:%S").second)
    dataset['day'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").day)
    dataset['month'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
    dataset['year'] = dataset['Date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").year)
    
    dataset['datetime'] = dataset.apply(lambda x: datetime.datetime(x['year'],
                                                                    x['month'],
                                                                    x['day'],
                                                                    x['hour'],
                                                                    x['minute'],
                                                                    x['second']),
                                                                    axis=1
                                                                    )

dataset_3['datetime'] = dataset_3.apply(lambda x: datetime.datetime.strptime(x.datetime, '%Y-%m-%dT%H:%M'), axis=1)
dataset_4['datetime'] = dataset_4.apply(lambda x: datetime.datetime.strptime(x.datetime, '%Y-%m-%dT%H:%M'), axis=1)

In [149]:
dataset_total = dataset_1.merge(dataset_2, how='left', on='datetime')
dataset_total = dataset_total.merge(dataset_3, how='left', on='datetime')
dataset_total = dataset_total.merge(dataset_4, how='left', on='datetime')


In [150]:
data = dataset_total

In [151]:
dataset_1.columns

Index(['Unnamed: 0', 'Tair', 'Tsoil', 'T_leaf', 'T_canopy', 'RH', 'ea', 'u',
       'fesc', 'fesc1', 'LAI', 'SWC', 'EF_daily', 'SW', 'PAR', 'APAR_canopy',
       'GPP_canopy', 'Reco_canopy', 'Rdtot', 'NEE_canopy', 'SIFcanopy_760nm',
       'LUE_canopy', 'SIFyield_canopy', 'Date', 'Time', 'APAR_label',
       'NEE_night', 'hour', 'minute', 'second', 'day', 'month', 'year',
       'datetime'],
      dtype='object')

In [152]:
dataset_2.columns

Index(['Unnamed: 0', 'Date', 'Time', 'Tair', 'RH', 'u', 'LAI', 'SWC', 'PAR',
       'VPD', 'GPP_canopy', 'Reco_canopy', 'NEE_canopy', 'SIFcanopy_760nm',
       'APAR_label', 'NEE_night', 'SIF_NN', 'NEE_NN', 'GPP_NN', 'Reco_NN',
       'Reco_NT', 'Reco_DT', 'GPP_NT', 'GPP_DT', 'hour', 'minute', 'second',
       'day', 'month', 'year', 'datetime'],
      dtype='object')

In [153]:
dataset_3.columns

Index(['datetime', 'year.year', 'month.month', 'doy.day',
       'hour.hour.est.utc.5.', 'seq_day.90.day.90', 'seq_time.90.decimal_day',
       'obs.nee.e.6mol.m2.s', 'obs.fco2.e.6mol.m2.s', 'fco2.corr.e.6mol.m2.s',
       'ustar.cm.s', 'nee.e.6mol.m2.s', 'resp.e.e.6mol.m2.s',
       'gee.e.6mol.m2.s', 'c.flag1.bit', 'c.flag2.bit', 'obs_ta_.27m.c',
       'ta.27m.filled.c', 'ta1.flag1.bit', 'ta1.flag2.bit', 'ta1.flag3.bit',
       'ta1.flag4.bit', 'ta.2.5m.c', 'ta.2.5m.filled.c', 'ta5.flag1.bit',
       'ta5.flag2.bit', 'ta5.flag3.bit', 'ta5.flag4.bit',
       'par.28m.e.6mol.m2.s', 'par.28m.filled.e.6mol.m2.s', 'p.flag1.bit',
       'p.flag2.bit', 'p.flag3.bit', 'p.flag4.bit', 'pardir_ue.m2.s',
       'pardfs_ue.m2.s'],
      dtype='object')

In [154]:
dataset_4.columns

Index(['datetime', 'year', 'month', 'seq.from.1990.days', 'doy', 'wind.sp',
       'wind.dir', 'eddy.t', 'eddy.q', 'o3.mlb', 'no.y', 's.no.y', 'co',
       's.co', 'f.co2', 'f.mom', 'f.heat', 'f.h20', 'f.o3', 'f.no.y', 'r.net',
       'par.12.7', 'par.29', 'rh.27.9m', 'rh.22.6m', 'rh.15.4m', 'rh.7.6m',
       'rh.2.5m', 'ta.27.9m', 'ta.22.6m', 'ta.15.4m', 'ta.7.6m', 'ta.2.5m',
       'ts.srfm', 'ts.50m', 'ts.20m', 'ts.a1.05cm', 'ts.a2.10cm', 'ts.a3.20cm',
       'ts.a4.50cm', 'ts.b1.05cm', 'ts.b2.20cm', 'ts.b3.50cm', 'ts.c1.05cm',
       'ts.c2.05cm', 'ts.c3.20cm', 'ts.c4.50cm', 'co2', 'co2.stg', 'p.amb',
       'ts.srf.adj', 'par.dir.ue.m2.s', 'par.dfs.ue.m2.s', 'par.ref.ue.m2.s',
       'fco2.7200', 'fh2o.7200', 'co2.7200', 'eddy.q.7200'],
      dtype='object')

In [290]:
data = dataset_total

In [291]:
mappings = {'Tair_x': 'TA',
            'LAI_x': 'LAI',
            'SWC_x': 'SWC',
            'PAR_x': 'SW_IN',
            'f.heat': 'LE',
            'wind.sp': 'WS',
            'wind.dir': 'WD',
            'p.amb': 'PA',
            'r.net': 'SW_IN_POT',
            'RH_x': 'RH',
            'EF_daily_x': 'EF_daily',
            'APAR_label_x': 'NIGHT',
            'year': 'Year',
            'NEE_canopy_x': 'NEE',
            'month_x': 'Month',
            'hour_x': 'Hour',
            'minute_x': 'Minute',
            'second_x': 'Second',
            'day_x': 'Day',
            'Reco_canopy_x': 'RECO',
            'GPP_canopy_x': 'GPP',
            'Reco_NN': 'RECO_NN',
            'Reco_DT': 'RECO_DT',
            'Reco_NT': 'RECO_NT'
            }

In [292]:
for old, new in mappings.items():
    #if old in data.columns and new not in data.columns:
    if old in data.columns:
        data[new] = data[old]

data['NIGHT'] = np.abs(data['NIGHT']-1)
data['Year'] = '2013'

In [293]:
# All Data Preparation
data = sw_pot_sm(data)
data = sw_pot_sm_diff(data)

data = data[['TA',
            'VPD',
            'Year',
            'Month',
            'Day',
            'Hour',
            'Minute',
            'datetime',
            'NEE',
            'SW_POT_sm',
            'SW_POT_sm_diff',
            'SW_IN',
            'SW_IN_POT',
            'SWC',
            'WS',
            'WD',
            'PA',
            'RH',
            'LAI',
            'EF_daily',
            'GPP',
            'RECO',
            'GPP_NN',
            'RECO_NN',
            'GPP_DT',
            'RECO_DT',
            'GPP_NT',
            'RECO_NT'
            ]]

data["doy"] = pd.to_datetime(data['datetime']).dt.dayofyear
#data = load_data(dataset_config["site_name"])
#data = unwrap_time(data)
data = standardize_column_names(data)

#data = comp_wdefcum(data)
data = diffuse_to_direct_rad(data)
#data = GPP_prox(data)
#data = WD_trans(data)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [294]:


W_var = list(set(dataset_config["W"]) & set(data.columns))
if len(list(set(dataset_config["W"]) - (set(W_var)))):
    print(
        f'For site {dataset_config["site_name"]} variables {list(set(dataset_config["W"]) - set(W_var))} in W not available'
    )

X_var = list(set(dataset_config["X"]) & set(data.columns))
if len(list(set(dataset_config["X"]) - (set(X_var)))):
    print(
        f'For site {dataset_config["site_name"]} variables {list(set(dataset_config["X"]) - set(X_var))} in X not available'
    )

# drop problematic rows
data.replace(-9999, np.nan)
data = data[~pd.isna(data[X_var + W_var + ["SW_IN", 'NEE', 'GPP', 'RECO', 'GPP_DT', 'GPP_NT', 'RECO_DT', 'RECO_NT']]).any(axis=1)]


data = normalize(data, X_var + W_var, dataset_config["norm_type"], masked_normalization=False)
if experiment_config["extrapolate"]:
    data_test = data.copy()
    data_test = normalize_test(data_test, X_var + W_var, dataset_config["norm_type"])

In [295]:
data['Month'].unique()

array([ 6,  7,  8,  9, 10,  5])

In [324]:
pre_computed=False
syn=True
relnoise=0
site = 'all'
dataset_config = dict(
            site_name = site,
            syn = syn,
            version = 'simple',
            Q10=1.5,
            relnoise=relnoise,
            pre_computed=pre_computed,
            transform_T = True,
            month_wise= False,
            moving_window=[3, 3],
            delta = 'heuristic3', #fixed value, max_heuristic, slope_heuristic 
            X = ['TA', 'SWC', 'RH', 'VPD', 'EF_daily', 'LAI', 'WS', 'WD'],
            #X = ['VPD', 'TA','wdefcum', 'SW_POT_sm', 'SW_POT_sm_diff', 'PA', 'WS', 'GPP_prox', 'P', 'WD_sin', 'WD_cos', 'NEE_nt_avg', 'TS_1', 'TS_2', 'SWC_1', 'SWC_2', 'SW_DIF'], #['VPD', 'TA'],#
            #X = ['VPD', 'TA', 'TS_1', 'TS_2', 'SWC_1', 'SWC_2'],
            W = [], #['SW_POT_sm', 'SW_POT_sm_diff'], #['wdefcum'],
            Q10_drivers = ['SW_IN'],
            years = 'all', # 'all'
            norm_type = 'standardize',
            Q10_fitting=False,
            Q10_include_SW=False,
            test_scenario=True,
            RMSE=True,
            alternative_fluxes=None,
            alternative_treatment=None, #"SW_IN_POT",
            )
    
#model_y_config = dict(model = 'GradientBoostingRegressor',
#                    min_samples_split = 5,
#                    min_samples_leaf = 40,
#                    max_depth=1,
#                    n_estimators=300,
#                    #learning_rate=0.1,
#                    #max_depth=3
#                    )

model_y_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 2,
                    min_samples_leaf = 1,
                    max_depth=3,
                    n_estimators=100,
                    #learning_rate=0.1,
                    #max_depth=3
                    )

model_t_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 2,
                    min_samples_leaf = 1,
                    max_depth=3,
                    n_estimators=100,
                    # max_depth=3
                    )
model_final_config = dict(model = 'GradientBoostingRegressor',
                    min_samples_split = 2,
                    min_samples_leaf = 1,
                    max_depth=3,
                    n_estimators=100)
                    #learning_rate=0.1,
                    #max_depth=3
dml_config = dict(cv=10)

model_configs = dict(y = model_y_config,
                    t = model_t_config,
                    final= model_final_config,
                    dml =dml_config)

experiment_config = dict(experiment_type = 'flux_partitioning',
                        comment='Flux partitioning',
                        extrapolate=False,
                        seed=1000)



In [303]:
Y

0       3.462620
1       3.361382
2       3.267783
3       3.188380
4       2.700788
          ...   
6571    1.632728
6572    1.661431
6573    1.681370
6574    1.671784
6575    1.627845
Name: NEE, Length: 6288, dtype: float64

In [330]:
model = GradientBoostingRegressor()

In [331]:
model.fit(X, T)

GradientBoostingRegressor()

In [332]:
model.score(X,T)

In [325]:
from dml4fluxes.experiments.utility import *

indices = data["Year"] == '2013'

if dataset_config["transform_T"]:
    data.loc[indices, "T"], parameter = transform_t(
        x=data.loc[indices, "SW_IN"],
        delta=dataset_config["delta"],
        data=data.loc[indices, :],
        month_wise=False,
        moving_window=dataset_config["moving_window"]
    )
T = data.loc[indices, "T"]
X = data.loc[indices, [var + "_n" for var in X_var]]

if not len(W_var):
    W = None
else:
    W = data.loc[indices, [var + "_n" for var in W_var]]

Y = data.loc[indices, "NEE"]

# prints the parameters of the hyperbolic light curve

0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-01  1.e-03 -2.e+00]
0
0
[ 1.e-

In [326]:
# Other estimators you can potentially use
# Model_configs['y']['model] needs the name as a string and the rest of the keys must match the parameters of the model

from econml.dml import SparseLinearDML, LinearDML, NonParamDML

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import clone
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import SGDRegressor

In [327]:
model_y = getattr(sys.modules[__name__], model_configs["y"]["model"])
config = model_configs["y"].copy()
del config["model"]
model_y = model_y(**config)

model_t = getattr(sys.modules[__name__], model_configs["t"]["model"])
config = model_t_config.copy()
del config["model"]
model_t = model_t(**config)

model_final = getattr(sys.modules[__name__], model_configs["final"]["model"])
config = model_final_config.copy()
del config["model"]
model_final = model_final(**config)

est = NonParamDML(
    model_y=model_y, model_t=model_t, model_final=model_final, **model_configs["dml"]
)

In [328]:
est.fit(Y=Y, X=X, W=W, T=T, cache_values=True)

In [329]:
est.__dict__

{'model_y': GradientBoostingRegressor(),
 'model_t': GradientBoostingRegressor(),
 'featurizer': None,
 'model_final': GradientBoostingRegressor(),
 'cv': 10,
 'discrete_treatment': False,
 'discrete_instrument': False,
 'random_state': None,
 'categories': 'auto',
 'mc_iters': None,
 'mc_agg': 'mean',
 '_ortho_learner_model_final': <econml.dml._rlearner._ModelFinal at 0x7f397a3fd7f0>,
 '_ortho_learner_model_nuisance': <econml.dml._rlearner._ModelNuisance at 0x7f397a3fd940>,
 '_d_y': (),
 '_d_t': (),
 '_input_names': {'feature_names': ['EF_daily_n',
   'VPD_n',
   'RH_n',
   'SWC_n',
   'WD_n',
   'LAI_n',
   'TA_n',
   'WS_n'],
  'output_names': ['NEE'],
  'treatment_names': ['T']},
 '_d_t_in': (),
 '_random_state': RandomState(MT19937) at 0x7F39F5A77940,
 '_d_x': (8,),
 '_d_w': None,
 '_d_z': None,
 'transformer': None,
 'z_transformer': None,
 '_models_nuisance': [[<econml.dml._rlearner._ModelNuisance at 0x7f397a3acd90>,
 'nuisance_scores_': ([[0.7088616394471465,
    0.692688065551

In [238]:
def gpp(x, t):
    return -est.const_marginal_effect(x) * t


def reco(x, t, w=None, y=None):
    return np.mean(
        [
            est._models_nuisance[0][i]._model_y.predict(x, w)
            - est.const_marginal_effect(x)
            * (est._models_nuisance[0][i]._model_t.predict(x, w))
            for i in range(len(est._models_nuisance[0]))
        ],
        axis=0,
    )
    # return np.mean([est._models_nuisance[0][i]._model_y.predict(x, w)-est.const_marginal_effect(x)*t for i in range(len(est._models_nuisance[0]))], axis=0)
    # return np.mean([y-est.const_marginal_effect(x)*(est._models_nuisance[0][i]._model_t.predict(x, w)) for i in range(len(est._models_nuisance[0]))], axis=0)


def reco_res(nee, x, t):
    return nee + gpp(x, t)


def nee(x, t, w=None):
    return -gpp(x, t) + reco(x, t, w)


def t(x, w=None):
    return np.mean(
        [
            est._models_nuisance[0][i]._model_t.predict(x, w)
            for i in range(len(est._models_nuisance[0]))
        ],
        axis=0,
    )


def lue(x):
    return -est.const_marginal_effect(x)

In [239]:
data['GPP_orth'] = gpp(X, T)
data['RECO_orth'] = reco(X, T)
data['NEE_orth'] = nee(X, T)


X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without feature names
X has feature names, but GradientBoostingRegressor was fitted without featur

In [240]:
mean_squared_error(data['GPP'], data['GPP_DT']) 

In [202]:
mean_squared_error(data['NEE'], data['NEE_NN']) 

KeyError: 'NEE_NN'

In [225]:
mean_squared_error(data['NEE'], data['NEE_orth']) 